In [1]:
import pandas as pd
import time
from collections import defaultdict
from itertools import product
import pickle

SHW = 100000

In [2]:
# g = pd.read_csv("data/attendances_over_time.csv.gz", encoding='latin-1', 
#                 header=None, names=["cust_id","age_gr","mtype_sec", "n_attend"])
# g.head()

In [3]:
mtypes_over_years = (pd.read_csv("data/ooo.csv.gz", encoding='latin-1', sep='\t')
                    .applymap(lambda x: x.split(",") if isinstance(x,str) else x)
                    )

In [4]:
mtypes_over_years.head()

,cust_id,00-17,18-24,25-29,30-34,35-39,40-44,45-49,50-54,55-59,60-64,65-69,70-74,75-84
0,11171,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[RUNION],NaN,NaN,NaN,NaN,NaN
1,11284,NaN,NaN,NaN,NaN,NaN,[HIPHOP],NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,11228,NaN,NaN,NaN,NaN,NaN,"[ALTERN, LEAGUE]",NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,11640,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[RUNION, LEAGUE]",[LEAGUE],NaN,NaN,NaN,NaN
4,11197,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[RUNION],NaN,NaN,NaN,NaN,NaN


In [16]:
v = mtypes_over_years.iloc[3000000:4000000,:].T

In [17]:
v.shape

(14, 1000000)

In [18]:
v.head()

,3000000,3000001,3000002,3000003,3000004,3000005,3000006,3000007,3000008,3000009,...,3999990,3999991,3999992,3999993,3999994,3999995,3999996,3999997,3999998,3999999
cust_id,12507050,12507063,20071087,11673401,12507064,11673451,13589823,12507067,12602873,13589840,...,16807677,14850567,14211464,15107842,14344475,15405545,14850574,14211514,15107859,14344490
00-17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[ROCK, SOCCER]",NaN,NaN,...,NaN,NaN,"[CIRCUS, POP, RAP, PUNK, COMEDY, 80SROCK]",NaN,NaN,NaN,NaN,"[HORSER, PLAY, SFMUSIC, AFL, RAP]",NaN,NaN
25-29,[ROCK],NaN,NaN,NaN,[RNB],NaN,NaN,"[80SROCK, NETBALL]",NaN,NaN,...,NaN,NaN,"[SOCCER, POP]",NaN,[POP],"[TENNIS, AFL, LEAGUE]",[CRICKET],NaN,[ALTERN],NaN
30-34,NaN,NaN,NaN,NaN,[80SROCK],NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,[CRICKET],NaN,[TENNIS],[CRICKET],NaN,NaN,NaN


In [19]:
def _cust_features(c):
    
    f = defaultdict(int)
    
    s = v.ix[:,c][v[c].notnull()]
    cid = s[0]
    
    if len(s) == 2:
        f = {"age_gr": s.index[1], "@"+s[1][0]: 1}
    elif len(s) == 3:
        f = {"age_gr": s.index[-1]}
        f.update({"@"+"->".join(tp): 1 for tp in product(s[1], s[2])})
    
    yield f

f = defaultdict(lambda: defaultdict(int))  # {cust_id: {"AFL->BALLET": 1, "TENNIS->COMEDY": 1}}

print("extracting customer features...")

t1 = time.time()

for c in range(3000000,4000000):
    for fch in _cust_features(c):
        f[v.ix[:,c][0]].update(fch)
        if (c+1)%SHW == 0:
            print("extracted features for {} customers...".format(c+1))

print("done. elapsed time {:.0f} min {:.0f} sec".format(*divmod(time.time() - t1, 60)))

pd.DataFrame.from_dict(f, orient="index").to_csv("p3.csv.gz",index=False, compression="gzip")
print("saved data")

extracting customer features...
extracted features for 3100000 customers...
extracted features for 3200000 customers...
extracted features for 3300000 customers...
extracted features for 3400000 customers...
extracted features for 3500000 customers...
extracted features for 3600000 customers...
extracted features for 3700000 customers...
extracted features for 3800000 customers...
extracted features for 3900000 customers...
extracted features for 4000000 customers...
done. elapsed time 17 min 18 sec
saved data
